In [24]:
import os
import numpy as np
import pandas as pd
from collections import Counter
from itertools import islice
import os.path

def words_generator(fileobj):
        for line in fileobj:
            for word in line.split():
                yield word
                
def generate_freq_dictionaries(root_dir):
        word_count_dict={}
        file_dict = {}
        emails_dirs = [os.path.join(root_dir,f) for f in os.listdir(root_dir)]    
        for emails_dir in emails_dirs:
            dirs = [os.path.join(emails_dir,f) for f in os.listdir(emails_dir)]
            for d in dirs:
                emails = [os.path.join(d,f) for f in os.listdir(d)]            
                for mail in emails:
                    file_name = mail.rsplit('/', 1)[-1]
                    file_dict[file_name] = {}
                    # if email is spam, assign spam label
                    if mail.split(".")[-1] == 'spam':
                        file_dict[file_name]["is_spam"] = 1 # spam
                    else:
                        file_dict[file_name]["is_spam"] = 0 # ham                    
                    f = open(mail,"r")
                    words = words_generator(f)
                    for word in words:
                        # if word is not an alpha numeric, don't include it
                        if word.isalpha()==False:
                            continue                       
                        if word not in word_count_dict:
                              word_count_dict[word] = 0
                        word_count_dict[word]+= 1                      
                        if word not in file_dict[file_name]:
                            file_dict[file_name][word] = 0
                        file_dict[file_name][word]+=1
        return word_count_dict,file_dict

In [25]:
 def find_most_frequent_words(word_count_dict):
        df = pd.DataFrame(word_count_dict.items(), columns=['word', 'total_freq'])
        df = df.sort(['total_freq'],ascending=False).head(n=3000)
        return df['word'].tolist(), df['total_freq'].tolist()

In [26]:
def get_freq(word,freq_dict):
        if word in freq_dict:
            return freq_dict[word]
        else:
            return 0

In [27]:
email_data_dir = "C:\\Users\\sarra\\Downloads\\RESEARCH\\SpamFiltering-master\\SpamFiltering-master\\enronEmailData"
word_count_dict,file_dict = generate_freq_dictionaries(email_data_dir)
freq_words,frequencies = find_most_frequent_words(word_count_dict)

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\sarra\\Downloads\\RESEARCH\\SpamFiltering-master\\SpamFiltering-master\\enronEmailData\\enron1\\enron1\\ham'

In [13]:
file_name = "C:\\Users\\sarra\\Downloads\\RESEARCH\\SpamFiltering-master\\SpamFiltering-master\\features.csv"

def extract_features(file_dict,freq_words,file_name): 
        features_df = pd.DataFrame(file_dict.items(), columns=['file_name', 'freq_values'])
        features_df['is_spam'] = features_df['freq_values'].apply(lambda x: x.get('is_spam'))
        for word in freq_words:
                features_df[word] = features_df['freq_values'].apply(lambda freq_dict: get_freq(word,freq_dict))
        columns = list(features_df.columns.values)
        columns.remove('freq_values')
        if os.path.isfile(file_name):
            with open(fname, 'a') as f:
                features_df.to_csv(f, columns =columns, header=False)
        else:
            features_df.to_csv(file_name, columns = columns,header=columns, index=None,sep='\t')
        del features_df
        
def chunks_generator(data, SIZE):
        it = iter(data)
        for i in xrange(0, len(data), SIZE):
            yield {k:data[k] for k in islice(it, SIZE)}

chunks_list = []
    # create chunks of file dictionary data and extract features preocessing each chunk iteratively
for item in chunks_generator(file_dict, 30):
    extract_features(item,freq_words,file_name)
            


NameError: name 'file_dict' is not defined